### revit and autocad project usage

In [1]:
# open source Python module that makes accessing ODBC databases simple
import pyodbc

# open source library written for data manipulation and analysis
import pandas as pd

# NumPy is a library for support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
import numpy as np

# Python3 code to demonstrate
# getting numbers from string 
# using re.findall()
import re

#exploratory data analysis
#from pandas_profiling import ProfileReport

In [2]:
sql_conn_creds = {"Server":"SOMESQL", "UID":"MY USERNAME", "PWD":"MY PASSWORD"}

In [3]:
def df_to_csv(dataframe, csv_name):
    csv_name = "C:\\Users\\daltongoodwin\\OneDrive - Henderson Engineers Inc\\Documents\\" + csv_name+".csv"
    dataframe.to_csv(csv_name, index=False)
    print("Saved")

In [4]:
def db_tables(database, sql_conn_creds):
    sql_conn = pyodbc.connect("Trusted_Connection=yes;" + 'Driver={SQL Server};Server=' + sql_conn_creds.get('Server') + ';Database=master;UID=' + sql_conn_creds.get('UID') + ';PWD=' + sql_conn_creds.get('PWD') + ';')
    cursor = sql_conn.cursor()
    tables_l = []
    for row in cursor.tables():
        tables_l.append(row.table_name)
    return tables_l

In [5]:
#db_tables("BIMData", sql_conn_creds)

In [6]:
def single_db_table(database, table, sql_conn_creds):
    sql_conn = pyodbc.connect("Trusted_Connection=yes;" + 'Driver={SQL Server};Server=' + sql_conn_creds.get('Server') + ';Database=master;UID=' + sql_conn_creds.get('UID') + ';PWD=' + sql_conn_creds.get('PWD') + ';')
    query = "SELECT * FROM " + database + ".dbo." + table
    cursor = sql_conn.cursor()
    return pd.read_sql(query, sql_conn)

In [7]:
def transpose(lst):
    return list(map(list, zip(*lst)))

In [8]:
#Creating the dataframe from the BIMData UsageData table. This data contains domain user name
df = single_db_table("BIMData", "UsageData", sql_conn_creds)

In [9]:
df = df[(df['LogDate'] > '2021-07-31 00:00:00')]

In [10]:
df = df[(df['Operation'] == 'qsave') | (df['Operation'] == 'save') | (df['Operation'] == 'saveas') | (df['Operation'] == 'savetocentral')]

In [11]:
doc_path = df['DocumentPath'].to_list()
project_num = df['ProjectNumber'].to_list()
revit_version = df['RevitVersion'].to_list()

In [12]:
#pars doc path string and find project number only if there is no project number
numbers_from_path = []
for path, num in zip(doc_path, project_num):
    if num == None:
        temp = re.findall(r'\d+', path)
        res = list(map(str, temp))
        if len(res) > 0:
            if len(max(res)) > 6:
                try:
                    numbers_from_path.append(int(max(res)))
                except:
                    numbers_from_path.append(max(res))
            else: numbers_from_path.append(None)
        else: numbers_from_path.append(None)
    else:
        try:
            numbers_from_path.append(int(num))
        except:
            numbers_from_path.append(num)

In [13]:
#group revit versions and AutoCAD
new_revit_version = []
for software in revit_version:
    if "revit" in software.lower():
        new_revit_version.append("Revit")
    else: new_revit_version.append("AutoCAD")

In [14]:
#combines project number and software version into single string
combined_num_and_software = []
for num, software in zip(numbers_from_path, new_revit_version):
    combined_num_and_software.append(str(num) + ", " + software)

In [15]:
#get rid of the vars that have "None" in it
cleaned_combined_num_and_software = []
for i in combined_num_and_software:
    if "None" not in i:
        cleaned_combined_num_and_software.append(i)

In [16]:
revit_software = []
autocad_software = []
combined_num_and_software
for i in set(cleaned_combined_num_and_software):
    if "AutoCAD" in i:
        autocad_software.append(i + ", " + str(combined_num_and_software.count(i)))
    else: revit_software.append(i + ", " + str(combined_num_and_software.count(i)))

In [17]:
project_nums = []
for i in autocad_software:
    project_nums.append(i.split(", ")[0])
for i in revit_software:
    project_nums.append(i.split(", ")[0])
project_nums = set(project_nums)

In [18]:
projects_and_ver_count = []
for num in project_nums:
    l = [num,0,0]
    for i in revit_software:
        if num in i:
            if i.split(", ")[1] == "Revit":
                l[1] = i.split(", ")[2]
            elif i.split(", ")[1] == "AutoCAD":
                l[2] = i.split(", ")[2]
    projects_and_ver_count.append(l)

In [19]:
project_revit_or_autocad = [[], []]
for i in projects_and_ver_count:
    project_revit_or_autocad[0].append(i[0])
    if int(i[1]) > int(i[2]):
        project_revit_or_autocad[1].append("Revit")
    else:
        project_revit_or_autocad[1].append("AutoCAD")       

In [20]:
project_revit_or_autocad = transpose(project_revit_or_autocad)

In [21]:
df = pd.DataFrame.from_records(project_revit_or_autocad)
df = df.rename(columns = {0:'ProjectNumber', 1:'Software'})

In [22]:
df_to_csv(df, "project_software_version")

Saved


In [1]:
#try to use exsisting j or bim 360 model paths to update the C: cached/working Revit files  